## Advanced tuning of parameters

In this tutorial, we will apply the skills from previous tutorials and build a classifier using the `Pipelines` and `FeatureUnion` functions from sklearn.

In [1]:
# IMPORT PACKAGES
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

### Data

We will be building a binary classifier that predicts wheter a person has diabetes or not, using information on the patient's health. 

The data can be found [here](https://drive.google.com/file/d/1TvCKlmH3Z32XAKk-VUcZyYu95Ccyw3PO/view?usp=sharing). 


In [2]:
#col_names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']

data = pd.read_csv("diabetes.csv", sep =';')

In [3]:
data.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


### Task

Build classifier which predicts the target variable `class` using rest of the attributes. The model should be fitted using pipeline that contains:
- PCA method
- SelectKBest method
- FeatureUnion
- Random Forest

Choose the best set of parameters using `Pipeline` and grid_search.

> #### Note
> **In this exercise, we are focusing on the implementation of pipeline. Since we have only 9 columns in our dataset the PCA is probably not the best technique to use during the data preparation from a methodology point of view.**

In [4]:
#select X and Y
X = data.drop('class', axis = 1)
y = data['class']

#split X and Y into train/test
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.8, random_state = 42)

In [5]:
#extraction techniques
pca = PCA(n_components = 2)
selection = SelectKBest(k=3)

#instantize classifier
rdf = RandomForestClassifier(random_state=123)

In [6]:
rdf.get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

## Approach 1

In [8]:
n_features_to_test = np.arange(1,9)

params = [
        {'reduce_dim': [PCA()],
         'reduce_dim__n_components': n_features_to_test,\
          "clf__n_estimators": [1,2,3],
         "clf__criterion":['gini','entropy','rbf'],
         "clf__max_depth":[2,3]},

        {'reduce_dim': [SelectKBest()],
         'reduce_dim__k': n_features_to_test,\
        "clf__n_estimators": [1,2,3],
         "clf__criterion":['gini','entropy','rbf'],
         "clf__max_depth":[2,3]}
]

In [9]:
pipe = Pipeline([('reduce_dim', PCA()),
                ("clf", rdf)
                ])

gridsearch = GridSearchCV(pipe, params, verbose=1).fit(X_train, y_train)



Fitting 5 folds for each of 288 candidates, totalling 1440 fits


In [10]:
print('Best params:', gridsearch.best_params_)
print('Final score is: ', gridsearch.score(X_test, y_test))

Best params: {'clf__criterion': 'gini', 'clf__max_depth': 3, 'clf__n_estimators': 3, 'reduce_dim': PCA(n_components=5), 'reduce_dim__n_components': 5}
Final score is:  0.6991869918699187


## Approach 2

In [11]:
#build a transformer
combined_features = FeatureUnion([("pca", pca),
                                 ("univ_select", selection)])

#create pipeline
pipeline = Pipeline([("features", combined_features),
                    ("clf", rdf)])

In [12]:
#set up param grid
params = {"features__pca__n_components": [1,2,3,4],
         "features__univ_select__k": [1,2,3,4],
         "clf__n_estimators": [1,2,3,4,5],
         "clf__criterion":['gini','entropy','rbf'],
         "clf__max_depth":[2,3,4,5]
         }

In [13]:
#create grid search object
grid_search = GridSearchCV(pipeline, params, verbose = 10, refit = True)

#fit model and tune params
grid_search.fit(X_train,y_train)

Fitting 5 folds for each of 960 candidates, totalling 4800 fits
[CV 1/5; 1/960] START clf__criterion=gini, clf__max_depth=2, clf__n_estimators=1, features__pca__n_components=1, features__univ_select__k=1
[CV 1/5; 1/960] END clf__criterion=gini, clf__max_depth=2, clf__n_estimators=1, features__pca__n_components=1, features__univ_select__k=1;, score=0.645 total time=   0.0s
[CV 2/5; 1/960] START clf__criterion=gini, clf__max_depth=2, clf__n_estimators=1, features__pca__n_components=1, features__univ_select__k=1
[CV 2/5; 1/960] END clf__criterion=gini, clf__max_depth=2, clf__n_estimators=1, features__pca__n_components=1, features__univ_select__k=1;, score=0.742 total time=   0.0s
[CV 3/5; 1/960] START clf__criterion=gini, clf__max_depth=2, clf__n_estimators=1, features__pca__n_components=1, features__univ_select__k=1
[CV 3/5; 1/960] END clf__criterion=gini, clf__max_depth=2, clf__n_estimators=1, features__pca__n_components=1, features__univ_select__k=1;, score=0.645 total time=   0.0s
[CV

[CV 4/5; 6/960] END clf__criterion=gini, clf__max_depth=2, clf__n_estimators=1, features__pca__n_components=2, features__univ_select__k=2;, score=0.667 total time=   0.0s
[CV 5/5; 6/960] START clf__criterion=gini, clf__max_depth=2, clf__n_estimators=1, features__pca__n_components=2, features__univ_select__k=2
[CV 5/5; 6/960] END clf__criterion=gini, clf__max_depth=2, clf__n_estimators=1, features__pca__n_components=2, features__univ_select__k=2;, score=0.667 total time=   0.0s
[CV 1/5; 7/960] START clf__criterion=gini, clf__max_depth=2, clf__n_estimators=1, features__pca__n_components=2, features__univ_select__k=3
[CV 1/5; 7/960] END clf__criterion=gini, clf__max_depth=2, clf__n_estimators=1, features__pca__n_components=2, features__univ_select__k=3;, score=0.613 total time=   0.0s
[CV 2/5; 7/960] START clf__criterion=gini, clf__max_depth=2, clf__n_estimators=1, features__pca__n_components=2, features__univ_select__k=3
[CV 2/5; 7/960] END clf__criterion=gini, clf__max_depth=2, clf__n_e

[CV 1/5; 13/960] END clf__criterion=gini, clf__max_depth=2, clf__n_estimators=1, features__pca__n_components=4, features__univ_select__k=1;, score=0.548 total time=   0.0s
[CV 2/5; 13/960] START clf__criterion=gini, clf__max_depth=2, clf__n_estimators=1, features__pca__n_components=4, features__univ_select__k=1
[CV 2/5; 13/960] END clf__criterion=gini, clf__max_depth=2, clf__n_estimators=1, features__pca__n_components=4, features__univ_select__k=1;, score=0.613 total time=   0.0s
[CV 3/5; 13/960] START clf__criterion=gini, clf__max_depth=2, clf__n_estimators=1, features__pca__n_components=4, features__univ_select__k=1
[CV 3/5; 13/960] END clf__criterion=gini, clf__max_depth=2, clf__n_estimators=1, features__pca__n_components=4, features__univ_select__k=1;, score=0.774 total time=   0.0s
[CV 4/5; 13/960] START clf__criterion=gini, clf__max_depth=2, clf__n_estimators=1, features__pca__n_components=4, features__univ_select__k=1
[CV 4/5; 13/960] END clf__criterion=gini, clf__max_depth=2, c

[CV 3/5; 19/960] END clf__criterion=gini, clf__max_depth=2, clf__n_estimators=2, features__pca__n_components=1, features__univ_select__k=3;, score=0.613 total time=   0.0s
[CV 4/5; 19/960] START clf__criterion=gini, clf__max_depth=2, clf__n_estimators=2, features__pca__n_components=1, features__univ_select__k=3
[CV 4/5; 19/960] END clf__criterion=gini, clf__max_depth=2, clf__n_estimators=2, features__pca__n_components=1, features__univ_select__k=3;, score=0.633 total time=   0.0s
[CV 5/5; 19/960] START clf__criterion=gini, clf__max_depth=2, clf__n_estimators=2, features__pca__n_components=1, features__univ_select__k=3
[CV 5/5; 19/960] END clf__criterion=gini, clf__max_depth=2, clf__n_estimators=2, features__pca__n_components=1, features__univ_select__k=3;, score=0.833 total time=   0.0s
[CV 1/5; 20/960] START clf__criterion=gini, clf__max_depth=2, clf__n_estimators=2, features__pca__n_components=1, features__univ_select__k=4
[CV 1/5; 20/960] END clf__criterion=gini, clf__max_depth=2, c

[CV 3/5; 25/960] END clf__criterion=gini, clf__max_depth=2, clf__n_estimators=2, features__pca__n_components=3, features__univ_select__k=1;, score=0.677 total time=   0.0s
[CV 4/5; 25/960] START clf__criterion=gini, clf__max_depth=2, clf__n_estimators=2, features__pca__n_components=3, features__univ_select__k=1
[CV 4/5; 25/960] END clf__criterion=gini, clf__max_depth=2, clf__n_estimators=2, features__pca__n_components=3, features__univ_select__k=1;, score=0.700 total time=   0.0s
[CV 5/5; 25/960] START clf__criterion=gini, clf__max_depth=2, clf__n_estimators=2, features__pca__n_components=3, features__univ_select__k=1
[CV 5/5; 25/960] END clf__criterion=gini, clf__max_depth=2, clf__n_estimators=2, features__pca__n_components=3, features__univ_select__k=1;, score=0.767 total time=   0.0s
[CV 1/5; 26/960] START clf__criterion=gini, clf__max_depth=2, clf__n_estimators=2, features__pca__n_components=3, features__univ_select__k=2
[CV 1/5; 26/960] END clf__criterion=gini, clf__max_depth=2, c

...

[CV 4/5; 942/960] END clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=4, features__pca__n_components=4, features__univ_select__k=2;, score=nan total time=   0.0s
[CV 5/5; 942/960] START clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=4, features__pca__n_components=4, features__univ_select__k=2
[CV 5/5; 942/960] END clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=4, features__pca__n_components=4, features__univ_select__k=2;, score=nan total time=   0.0s
[CV 1/5; 943/960] START clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=4, features__pca__n_components=4, features__univ_select__k=3
[CV 1/5; 943/960] END clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=4, features__pca__n_components=4, features__univ_select__k=3;, score=nan total time=   0.0s
[CV 2/5; 943/960] START clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=4, features__pca__n_components=4, features__univ_select__k=3
[CV 2/5; 943/960] END clf__criterion=rbf, clf__max_depth=5, clf__n_

[CV 5/5; 948/960] END clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=5, features__pca__n_components=1, features__univ_select__k=4;, score=nan total time=   0.0s
[CV 1/5; 949/960] START clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=5, features__pca__n_components=2, features__univ_select__k=1
[CV 1/5; 949/960] END clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=5, features__pca__n_components=2, features__univ_select__k=1;, score=nan total time=   0.0s
[CV 2/5; 949/960] START clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=5, features__pca__n_components=2, features__univ_select__k=1
[CV 2/5; 949/960] END clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=5, features__pca__n_components=2, features__univ_select__k=1;, score=nan total time=   0.0s
[CV 3/5; 949/960] START clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=5, features__pca__n_components=2, features__univ_select__k=1
[CV 3/5; 949/960] END clf__criterion=rbf, clf__max_depth=5, clf__n_

[CV 1/5; 955/960] END clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=5, features__pca__n_components=3, features__univ_select__k=3;, score=nan total time=   0.0s
[CV 2/5; 955/960] START clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=5, features__pca__n_components=3, features__univ_select__k=3
[CV 2/5; 955/960] END clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=5, features__pca__n_components=3, features__univ_select__k=3;, score=nan total time=   0.0s
[CV 3/5; 955/960] START clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=5, features__pca__n_components=3, features__univ_select__k=3
[CV 3/5; 955/960] END clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=5, features__pca__n_components=3, features__univ_select__k=3;, score=nan total time=   0.0s
[CV 4/5; 955/960] START clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=5, features__pca__n_components=3, features__univ_select__k=3
[CV 4/5; 955/960] END clf__criterion=rbf, clf__max_depth=5, clf__n_

[CV 4/5; 960/960] START clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=5, features__pca__n_components=4, features__univ_select__k=4
[CV 4/5; 960/960] END clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=5, features__pca__n_components=4, features__univ_select__k=4;, score=nan total time=   0.0s
[CV 5/5; 960/960] START clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=5, features__pca__n_components=4, features__univ_select__k=4
[CV 5/5; 960/960] END clf__criterion=rbf, clf__max_depth=5, clf__n_estimators=5, features__pca__n_components=4, features__univ_select__k=4;, score=nan total time=   0.0s


GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('pca',
                                                                        PCA(n_components=2)),
                                                                       ('univ_select',
                                                                        SelectKBest(k=3))])),
                                       ('clf',
                                        RandomForestClassifier(random_state=123))]),
             param_grid={'clf__criterion': ['gini', 'entropy', 'rbf'],
                         'clf__max_depth': [2, 3, 4, 5],
                         'clf__n_estimators': [1, 2, 3, 4, 5],
                         'features__pca__n_components': [1, 2, 3, 4],
                         'features__univ_select__k': [1, 2, 3, 4]},
             verbose=10)

In [14]:
print(grid_search.best_params_)

{'clf__criterion': 'entropy', 'clf__max_depth': 3, 'clf__n_estimators': 2, 'features__pca__n_components': 2, 'features__univ_select__k': 2}


In [15]:
#make new pipeline with best params
pipe = Pipeline(steps=[('pca', PCA(n_components=2)),
                        ('select_best', SelectKBest(k=2)),
                       ('classifier', RandomForestClassifier(n_estimators=2, 
                                                     criterion = 'entropy', 
                                                     max_depth = 3,
                                                     random_state=123))
                ])

In [16]:
#train and test
pipe = pipe.fit(X_train, y_train)
print('Testing score: ', pipe.score(X_test, y_test))

Testing score:  0.6829268292682927


In [17]:
from sklearn.metrics import accuracy_score
y_pred = pipe.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.6829268292682927


It is better without the PCA in the Pipeline
